In [1]:
import numpy as np
import tensorflow as tf
import os
#from riptide.models.cifar_resnet import cifar_resnet20_v1
#from riptide.binary.models.q_cifar_resnet import cifar_resnet20_v1
from riptide.models.vgg11 import vgg11
from riptide.binary.binary_funcs import XQuantize, HWGQuantize, load_clusters, load_bits, DQuantize, Quantize
from riptide.binary.binary_layers import Config
from riptide.utils.thread_helper import setup_gpu_threadpool
from slim.preprocessing.cifarnet_preprocessing import preprocess_image
from research.object_detection.utils.learning_schedules import exponential_decay_with_burnin

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train
y_train = y_train.astype(np.int32)
x_test = x_test
y_test = y_test.astype(np.int32)

In [4]:
batch_size = 100
num_gpus = 1
op_threads, num_workers = setup_gpu_threadpool(num_gpus)

def train_transform(data, label):
    #data = tf.image.convert_image_dtype(data, tf.float32)
    #data = tf.random_crop(data, [32,32,3])
    #data = tf.image.random_flip_left_right(data)
    #data = tf.image.random_brightness(data, 0.1)
    #data = tf.image.random_contrast(data, 0.9, 1.1)
    #data = tf.image.random_saturation(data, 0.9, 1.1)
    #data = tf.image.per_image_standardization(data)
    data = preprocess_image(data, 32, 32, is_training=True)
    return data, label

def test_transform(data, label):
    #data = tf.image.convert_image_dtype(data, tf.float32)
    #data = tf.image.resize_images(data, [32,32])
    #data = tf.image.per_image_standardization(data)
    data = preprocess_image(data, 32, 32, is_training=False)
    return data, label

def train_input_fn():
    ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    ds = ds.prefetch(batch_size)
    ds = ds.shuffle(5000)
    ds = ds.repeat(120)
    ds = ds.map(train_transform, num_parallel_calls=num_workers)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.contrib.data.AUTOTUNE)
    return ds

def eval_input_fn():
    ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    ds = ds.map(test_transform, num_parallel_calls=num_workers)
    ds = ds.batch(batch_size)
    ds = ds.repeat(1)
    return ds

INFO:tensorflow:Logical CPU cores: 12
INFO:tensorflow:TF_GPU_THREAD_COUNT: 1
INFO:tensorflow:TF_GPU_THREAD_MODE: gpu_private


In [5]:
normal = False

def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    if normal:
        actQ = None
        weightQ = None
        bits = None
        use_act = True
        use_bn = True
        use_maxpool = False
    else:
        actQ = DQuantize
        weightQ = XQuantize
        bits = 2.0#load_clusters(2)
        use_act = False
        use_bn = False
        use_maxpool = False
        pure_shiftnorm = True
    config = Config(actQ=actQ, weightQ=weightQ, bits=bits, use_act=use_act, use_bn=use_bn, use_maxpool=use_maxpool, pure_shiftnorm=pure_shiftnorm)

    with config:
        model = vgg11(classes=10)
    
    # Get proper mode for batchnorm and dropout, must be python bool.
    if mode == tf.estimator.ModeKeys.TRAIN:
        is_training = True
    else:
        is_training = False
        
    logits = model(features, training=is_training)
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    def exclude_batch_norm(name):
        return (('batch_normalization' not in name) and ('shift_normalization' not in name) and (('unbinarized' in name) or normal))
    
    weight_decay = 1e-4
    l2_loss = weight_decay * tf.add_n(
        # loss is computed using fp32 for numerical stability.
        [tf.nn.l2_loss(tf.cast(v, tf.float32)) for v in tf.trainable_variables()
        if exclude_batch_norm(v.name)])

    loss = l2_loss + cross_entropy
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        lr = tf.train.exponential_decay(0.01, tf.train.get_global_step(), 20000, 0.1)
        optimizer = tf.train.MomentumOptimizer(learning_rate = lr, momentum=0.9, use_nesterov=True)
        #optimizer = tf.train.AdamOptimizer(learning_rate = lr)
        # Need to specify update ops as updating, when inheriting from keras use get_updates_for.
        update_ops = model.get_updates_for(features)
        print(update_ops)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
# Create the Estimator
strategy = tf.contrib.distribute.MirroredStrategy(num_gpus=num_gpus)
session_config = tf.ConfigProto(
    inter_op_parallelism_threads=op_threads,
    intra_op_parallelism_threads=op_threads,
    allow_soft_placement=True)
run_config = tf.estimator.RunConfig(
    #train_distribute=strategy,
    session_config=session_config)
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
    config=run_config,
    model_dir="/data/jwfromm/models/vgg_dorefa_true_shiftnorm_confirm")
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=None)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

INFO:tensorflow:Using config: {'_model_dir': '/data/jwfromm/models/quick_test', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': intra_op_parallelism_threads: 11
inter_op_parallelism_threads: 11
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8736807400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
tf.estimator.train_and_evaluate(mnist_classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
[<tf.Operation 'vgg11/unbinarized/batch_normalization/AssignMovingAvg/AssignSubVariableOp' type=AssignSubVariableOp>, <tf.Operation 'vgg11/unbinarized/batch_normalization/AssignMovingAvg_1/AssignSubVariableOp' type=AssignSubVariableOp>, <tf.Operation 'vgg11/shift_normalization/AssignMovingAvg/AssignSubVariableOp' type=AssignSubVariableOp>, <tf.Operation 'vgg11/shift_normalization/AssignMovingAvg_1/AssignSubVariableOp' type=AssignSubVariableOp>, <tf.Operation 'vgg11/shift_normalization_1/AssignMovingAvg/AssignSubVariableOp' type=AssignSubVariableOp>, <tf.Operation 'vgg11/shift_normalization_1/AssignMovingAvg_1/Assig

KeyboardInterrupt: 

In [ ]:
mnist_classifier.evaluate(eval_input_fn)

In [ ]:
# HWGQ gets to around 81
# full precision gets to 89
# DQ no batchnorm + scalu only activation gets to 82.5
# 78 with shift mean
# 77 with binary output
# adding higher weight decay gives 80.5 full binary
# sSecond run gets to 78.3%, after fix. Makes it worse?

# CifarNet
# Full precision with batchnorm and relu gets to 85%
# no maxpool version at 82.5

# VGG
# Full precision gets to 87%, 86% with no maxpooling
# hwgq full binary gets to 81.8%
# dorefa gets to 84
# No max pool normal first layers gets to 82%, binary output doesnt effect accuracy
# Taking out mean shifting gives 81%
# shift norming gives 81%